In [ ]:
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
# from functools import reduce import json
import time
import pandas as pd
import string

def write_to_AzureStorage(df,file_name):
    from azure.storage.blob import (BlockBlobService)
    import pandas as pd
    import io
    output = io.StringIO()
    output = df.to_csv (encoding = "utf-8")
    # Give the appropriate details here.
#     accountName = "bdpproj1"
#     accountKey = "qDRRza9BkroEhjC8rOJcEZ70WU9fFbXOylwcjtWT6mjGd1AVZ9O2dgd+s0+m5vzL39Fix+TvxDwSABtvadjQEg=="
#     containerName = "new"
    accountName = "bdpcluster"
    accountKey = "s6rMMR0sNTFydkMHSbBftfS3tGgTD5wtF4MCUmDpsphqvwYyy0KF8TO/Bk05HL+73QCQXB0rkOTRB06YWElyTg=="
    containerName = "new"
    blobName = "ParallelReweets.csv"
    blobService = BlockBlobService(account_name=accountName, account_key=accountKey)
    blobService.create_blob_from_text('new',file_name, output)
    
def is_pokemon_tweet(text):
    # pokemon_words = ['pokemongo']
    key_words = ['pokemon', 'pokecoins', 'pokmon', 'twitter', 'facebook', 'tinder', 'snapchat', 
                 'amazon', 'clash', 'reigns', 'renegade', 'asphalt', 'pinout', 'sudoku', 'mario',
                 'dues', 'leagueoflegends', 'dota', 'templerun', 'flappybird']
    new_text = " ".join(filter(lambda x:x[0]!='@', text.split())).replace(' ', '').replace('é', 'e')
    for word in key_words:
        if word in new_text:
            return True
    return False

sqlContext = SQLContext(spark)
global sqlContext
wanted_fields = ['created_at', 'id', 'text', 'source', 'user', 'geo', 'coordinates', 'place', 
                 'retweet_count', 'favorite_count', 'entities', 'lang']
dataset_path = "wasbs:///anit/10/*/*/*"                                        # for reading from primary storage
# dataset_path = "wasb://new@bdpproj1.blob.core.windows.net/anit/10/01/01/*"      # for secondary storage 
# dataset_path = '../Dataset/OctData/*/*/*'                                       # for local storage
start_time = int(time.time()*1000.0)
df = sqlContext.read.json(dataset_path)
df = df.where('delete is null')
df = df.where('lang is not null')
mixed_tweets_df = df.where('retweeted_status is not null')
check_pokemon = udf(lambda y: is_pokemon_tweet(y), BooleanType())
mixed_tweets_df = mixed_tweets_df.where(check_pokemon('retweeted_status.text') == True)
nested_tweets_df = mixed_tweets_df.select('retweeted_status.*').select(wanted_fields)
retweets_df = mixed_tweets_df.selectExpr(wanted_fields + ['retweeted_status.id as retweeted_id'])
tweets_df = df.where('retweeted_status is null').where(check_pokemon('text') == True).select(wanted_fields)
final_tweets_df = tweets_df.union(nested_tweets_df)
print('Writing into CSV files...')
write_to_AzureStorage(final_tweets_df.toPandas(),"SparkAllTweets.csv")
write_to_AzureStorage(retweets_df.toPandas(),"SparkAllRetweets.csv")
end_time = int(time.time()*1000.0)
print('Running Time: ', (end_time - start_time)/1000.0, 'seconds')
